In [ ]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.display import display
from scipy import linalg, spatial, stats
from sklearn.linear_model import LinearRegression

from rp2 import get_data_path, get_output_path, GeneSymbolMap

Load the results for phagocytes from Hagai *et al.* (2018) and determine their list of "responsive" genes

In [ ]:
phagocyte_genes_df = pd.read_excel(
    get_data_path("hagai_2018", "41586_2018_657_MOESM4_ESM.xlsx"),
    sheet_name="phagocytes_FC_diveregnce"
)
display(phagocyte_genes_df)

In [ ]:
study_species = "mouse"

responsive_phagocyte_genes = phagocyte_genes_df.loc[phagocyte_genes_df[f"{study_species}_padj"] < 0.01].gene
print(f"{len(responsive_phagocyte_genes):,} responsive phagocyte genes")

Load Hagai *et al.* (2018) UMI counts and reduce the dataset to values of interest to optimise memory usage and access performance

In [ ]:
array_express_path = get_data_path("ArrayExpress")
umi_ad = anndata.read_h5ad(array_express_path.joinpath(f"E-MTAB-6754.processed.2.{study_species}.h5ad"))

study_genes = responsive_phagocyte_genes
study_replicates = ["1", "2", "3"]
study_control = "unst"
study_treatments = ["lps", "pic"]
study_time_points = ["0", "2", "4", "6"]

umi_ad = umi_ad[:, study_genes]
umi_ad = umi_ad[umi_ad.obs.replicate.isin(study_replicates), :]
umi_ad = umi_ad[umi_ad.obs.treatment.isin(study_treatments + [study_control]), :]
umi_ad = umi_ad[umi_ad.obs.time_point.isin(study_time_points), :]
umi_ad = umi_ad.copy()

Create a map between gene IDs and symbols

In [ ]:
gene_symbols_df = pd.read_table(
    get_data_path("BioMart", f"{study_species}_genes.tsv"),
    names=["id", "symbol", "description"],
    index_col=0
)
symbol_map = GeneSymbolMap(gene_symbols_df)

Create a rough estimation of gene variability

In [ ]:
gene_variability = pd.Series(index=umi_ad.var.index, data=umi_ad.X.A.var(axis=0)).sort_values(ascending=False)
print(gene_variability)

Create a StudyTreatmentSet for each treatment of interest so they may each be analysed

In [ ]:
class StudyTreatmentSet:
    def __init__(self, umi_counts_ad, treatments, control_name):
        all_treatments = treatments + [control_name]
        self.umi_counts_ad = umi_counts_ad[umi_counts_ad.obs.treatment.isin(all_treatments), :]


treatment_sets = {}
for treatment in study_treatments:
    treatment_sets[treatment] = StudyTreatmentSet(umi_ad, [treatment], study_control)

if len(study_treatments) > 1:
    treatment_sets["all"] = StudyTreatmentSet(umi_ad, study_treatments, study_control)

# Restrict focus to the LPS treatment for now
umi_ad = treatment_sets["lps"].umi_counts_ad

Calculate and display per condition statistics

In [ ]:
condition_stats_df = pd.DataFrame()

for (replicate, treatment, time_point), group_df in umi_ad.obs.groupby(["replicate", "treatment", "time_point"]):
    print(f"Calculating condition statistics for replicate={replicate}, treatment={treatment}, time={time_point}h")

    cell_view = umi_ad[group_df.index, :]

    condition_stats_df = condition_stats_df.append(
        pd.DataFrame(
            data={
                "gene": study_genes,
                "replicate": replicate,
                "treatment": treatment,
                "time_point": time_point,
                "n_barcodes": cell_view.n_obs,
                "min": cell_view.X.A.min(axis=0),
                "max": cell_view.X.A.max(axis=0),
                "mean": cell_view.X.A.mean(axis=0),
                "variance": cell_view.X.A.var(axis=0, ddof=1),
                "std_dev": cell_view.X.A.std(axis=0, ddof=1),
            },
        ),
        ignore_index=True,
    )

condition_stats_df = condition_stats_df.sort_values(["gene", "replicate", "time_point", "treatment"])
display(symbol_map.added_to(condition_stats_df))

Display conditions that have a zero mean UMI count

In [ ]:
zero_mean_df = condition_stats_df.loc[condition_stats_df["mean"] == 0]
display(symbol_map.added_to(zero_mean_df))
display(zero_mean_df.time_point.value_counts().sort_index())

Determine outliers based on (squared) Mahalanobis distance

In [ ]:
def calculate_mahalanobis_distance(df, column_name="distance"):
    centroid = df.mean(axis=0)
    cov_mtx = np.cov(df, rowvar=False)
    inv_cov_mtx = linalg.inv(cov_mtx)

    return pd.DataFrame(
        index=df.index,
        data={
            column_name: [spatial.distance.mahalanobis(row, centroid, inv_cov_mtx)
                          for row in df.to_numpy()],
        },
    )

In [ ]:
mahalanobis_distances = condition_stats_df.groupby("gene")[["mean", "variance"]].apply(calculate_mahalanobis_distance)

outlier_distance_threshold = np.sqrt(stats.chi2.ppf(0.95, 2))
condition_stats_df["m_distance"] = mahalanobis_distances.loc[condition_stats_df.index]
condition_stats_df["outlier"] = condition_stats_df.m_distance > outlier_distance_threshold

print(f"Outliers: {condition_stats_df.outlier.sum():,}")

Plot histograms of mean and variance for all non-outliers

In [ ]:
for stat_name in ["Mean", "Variance"]:
    condition_stats_df.loc[~condition_stats_df.outlier, stat_name.lower()].plot.hist(log=True).set(xlabel=stat_name)
    plt.show()

condition_stats_df.loc[~condition_stats_df.outlier].groupby("gene")["mean"].max().plot.hist(log=True).set(xlabel="Max mean")
plt.show()

Fit linear regression model to mean-variance relationship of all genes

In [ ]:
def fit_mean_var_regression(gene_df):
    mean_and_var = gene_df[["mean", "variance"]].to_numpy()
    outlier_mask = gene_df.outlier.to_numpy()

    lr = LinearRegression().fit(*mean_and_var[np.newaxis, ~outlier_mask].T)
    r2 = lr.score(*mean_and_var[np.newaxis, ~outlier_mask].T)

    return pd.Series(data={
        "slope": np.squeeze(lr.coef_),
        "intercept": np.squeeze(lr.intercept_),
        "r2": r2,
    })


gene_regression_df = condition_stats_df.groupby("gene").apply(fit_mean_var_regression)
display(symbol_map.added_to(gene_regression_df))

Save the descriptive statistics and results of fitting the regression models

In [ ]:
output_path = get_output_path()
output_path.mkdir(parents=True, exist_ok=True)

condition_stats_df.to_csv(output_path.joinpath(f"{study_species}_stats_per_condition_per_gene.csv"), index=False)
gene_regression_df.to_csv(output_path.joinpath(f"{study_species}_lr_fit_per_gene.csv"))

Show mean-variance plots for 10 most variable genes

In [ ]:
plot_outliers = True

replicate_colour_map = {
    "1": "black",
    "2": "red",
    "3": "blue",
}

for gene in gene_variability.index[:10]:
    gene_df = condition_stats_df.loc[condition_stats_df.gene == gene]

    slope, intercept = gene_regression_df.loc[gene, ["slope", "intercept"]]

    lr_plot_x = gene_df["mean"]
    if not plot_outliers:
        lr_plot_x = lr_plot_x[~gene_df.outlier]
    lr_plot_x = np.sort(lr_plot_x)
    lr_plot_y = (lr_plot_x * slope) + intercept

    ax = plt.subplot()
    if plot_outliers:
        gene_df.loc[gene_df.outlier].plot.scatter("mean", "variance", ax=ax, marker="x")

    for replicate, df in gene_df.loc[~gene_df.outlier].groupby("replicate"):
        df.plot.scatter("mean", "variance", ax=ax, marker="o", label=replicate, color=replicate_colour_map[replicate])
    plt.plot(lr_plot_x, lr_plot_y)
    plt.title(f"{symbol_map.lookup(gene)} / {gene}")
    plt.xlabel("Mean")
    plt.ylabel("Variance")
    plt.xlim(left=min(0, np.min(lr_plot_x)))
    plt.ylim(bottom=min(0, np.min(lr_plot_y)))
    plt.legend(title="Replicate")
    plt.show()